Centrala posiada uszkodzone zdjęcia odzyskane z aparatu cyfrowego. Istnieje szansa, że na niektórych z nich jest Barbara. Nie wiemy, jak wygląda Barbara. Możesz na żywo porozmawiać z automatem działającym w centrali. Automat nie jest zbyt sprytny, ale może Ci pomóc w poprawieniu jakości zdjęć i w naprawianiu ich. Twoim zadaniem jest przygotowanie rysopisu Barbary.

Automat może dla Ciebie poprawić posiadane zdjęcia. Obsługuje on kilka narzędzi:

naprawa zdjęcia zawierającego szumy/glitche

rozjaśnienie fotografii

przyciemnienie fotografii

Oto polecenia, które rozpoznaje automat:

REPAIR NAZWA_PLIKU

DARKEN NAZWA_PLIKU

BRIGHTEN NAZWA_PLIKU

Gdy będziesz mieć już pewność co do wyglądu Barbary, przygotuj jej rysopis w języku polskim. Uwzględnij wszystkie szczegóły ze zdjęć, które pomogą nam ją rozpoznać.

Zadanie nazywa się photos.

API do obróbki zdjęć działa w sposób opisany poniżej i słucha jak zawsze na /report

{
 "task":"photos",
 "apikey":"TWÓJ KLUCZ API",
 "answer":"START"
}

Słowem “START” rozpoczynasz rozmowę z automatem. Przedstawi Ci on cztery fotografie. Niekoniecznie wszystkie z nich przedstawiają Barbarę i nie wszystkie z nich zawierają istotne dla nas szczegóły. Wydaj automatowi polecenia, mówiąc, na którym zdjęciu powinien wykonać jaką operację.

Co należy zrobić w zadaniu?

Wyślij do zadania o nazwie “photos” pole “answer” ustawione na “START” (tylko to jedno słowo). Pamiętaj o przesłaniu swojego klucza API. To zapytanie wygląda dokładnie tak, jak przy każdym innym zgłaszaniu odpowiedzi.

W odpowiedzi na zapytanie, automat podzieli się z Tobą czterema zdjęciami odzyskanymi z uszkodzonego aparatu cyfrowego. Automat nie wie, co one przedstawiają. Na pewno na którymś, a może nawet na wszystkich jest Barbara.

Poinstruuj automat, jak należy poprawić te zdjęcia. Możesz wysyłać polecenia np: DARKEN IMG_1234.PNG

Automat odpowiada w języku naturalnym, zwracając nieustrukturyzowane dane. Czasami to będzie URL do zdjęcia, a czasami opis tego, jak to zdjęcie zdobyć.

Jeśli już wiesz, jak wygląda Barbara, przygotuj jej rysopis i wyślij go w polu “answer” do centrali jako zadanie “photos”. 

Jeśli eksperci pracujący w centrali potwierdzą, że rysopis jest wystarczający, to w odpowiedzi otrzymasz flagę.

UWAGA: każde poprawne polecenie wydane do automatu kończy się kodem ZERO i zwrotką “200 OK”. Nie oznacza to jednak, że zadanie jest zaliczone, a jedynie znaczy tyle, że poprawnie udało Ci się porozmawiać z automatem.

Format finalnej odpowiedzi na zadanie:

{
    "task": "photos",
    "apikey": "1111-11-11-11-1111",
    "answer": "tekstowy dokładny rysopis Barbary"
}

🧅 HINT 🧅: jeśli chcesz oszczędzić tokeny w tym zadaniu, to każde z dostarczonych zdjęć posiada wersję o 50% mniejszą. Wystarczy dopisać do nazwy pliku sufix “-small”, czyli zamiast IMG_123.PNG możesz operować na IMG_123-small.PNG. Pamiętaj, że na fotkach o niższej rozdzielczości, rozpoznawanie elementów może być trudniejsze dla modeli LLM.

In [1]:
import os
import requests
import json
from dotenv import load_dotenv

In [2]:
load_dotenv()

personal_api_key = os.getenv("PERSONAL_API_KEY")

In [5]:
querry_json_automat = {
    "task": "photos",
    "apikey": personal_api_key,
    "answer": "START"
}
automat_url = "https://centrala.ag3nts.org/report"
get_data_automat = requests.post(automat_url, json=querry_json_automat)
print(get_data_automat.json())

{'code': 0, 'message': 'No kogo ja widzę! Numer piąty!. Oto fotki, które udało nam się zdobyć. IMG_559.PNG, IMG_1410.PNG, IMG_1443.PNG, IMG_1444.PNG. Wszystkie siedzą sobie tutaj: https://centrala.ag3nts.org/dane/barbara/. Pamiętaj, że zawsze mogę\xa0poprawić\xa0je dla Ciebie (polecenia: REPAIR/DARKEN/BRIGHTEN).'}


In [8]:
# Define base URL and image filenames
base_url = "https://centrala.ag3nts.org/dane/barbara/"
image_files = ["IMG_559.PNG", "IMG_1410.PNG", "IMG_1443.PNG", "IMG_1444.PNG"]

# Download each image
for img_file in image_files:
    img_url = base_url + img_file
    response = requests.get(img_url)
    
    # Save the image if download successful
    if response.status_code == 200:
        with open(img_file, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {img_file}")
    else:
        print(f"Failed to download {img_file}")
# from IPython.display import Image, display

# # Display each downloaded image
# for img_file in image_files:
#     display(Image(filename=img_file))


Downloaded IMG_559.PNG
Downloaded IMG_1410.PNG
Downloaded IMG_1443.PNG
Downloaded IMG_1444.PNG


In [32]:
import re
def repair_image(image_file, command):
    querry_json_repair = {
        "task": "photos",
        "apikey": personal_api_key,
        "answer": command + " " + image_file
    }
    automat_url = "https://centrala.ag3nts.org/report"
    get_data_repair = requests.post(automat_url, json=querry_json_repair)
    print(get_data_repair.json())
    # Download the repaired image
    # Find the first .png file in the message
    message = get_data_repair.json()["message"]
    picture = re.search(r'[A-Za-z0-9_]+\.png', message, re.IGNORECASE)
    if picture:
        picture = picture.group(0)
    else:
        raise ValueError("No .png file found in message")
    repaired_img_url = "https://centrala.ag3nts.org/dane/barbara/" + picture
    response = requests.get(repaired_img_url)

    # Save the repaired image if download successful
    if response.status_code == 200:
        with open("./img/" + picture, 'wb') as f:
            f.write(response.content)
        print("Downloaded repaired image " + picture)
    else:
        print("Failed to download repaired image")

In [12]:
import openai
openai_api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=openai_api_key)

In [13]:
def encode_image_to_base64(image_path):
    import base64
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [37]:
def analyze_image(image_path: str):
    base64_image = encode_image_to_base64(image_path)

    # Create message with image
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": '''You get a image of Barbara and need to response with a name of the Barbara. 
                Barbara is a woman. 
                If picture is damaged reply with REPAIR. 
                To Dark reply with one word BRIGHTEN.
                To Brighten reply with one word DARKEN.
                If You can describe Barbara, reply with her description.
                think step by step. how she look describe details like hair, eyes, clothes, etc.
                what she is doing? what she have?
                '''
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    },
                    {
                        "type": "text", 
                        "text": "What is on this image?"
                    }
                ]
            }
        ],
        max_tokens=300
    )
    return response.choices[0].message.content

In [27]:
answer = analyze_image("./img/IMG_559_FGR4.PNG")
print(answer)
answer = answer.replace(".", "")

DARKEN


In [33]:
repair_image("IMG_559_FGR4.PNG", answer)

{'code': 0, 'message': 'Noooo! Teraz to wygląda naprawdę dobrze. Mamy ją! IMG_559_NRR7.PNG'}
Downloaded repaired image IMG_559_NRR7.PNG


In [40]:
answer = analyze_image("./img/IMG_1410.PNG")
print(answer)
answer = answer.replace(".", "")
repair_image("IMG_1410.PNG", answer)

BRIGHTEN
{'code': 0, 'message': 'NO! Teraz widać\xa0twarze i włosy. To był dobry pomysł! https://centrala.ag3nts.org/dane/barbara/IMG_1410_FXER.PNG'}
Downloaded repaired image IMG_1410_FXER.PNG


In [41]:
answer = analyze_image("./img/IMG_1443.PNG")
print(answer)
answer = answer.replace(".", "")
repair_image("IMG_1443.PNG", answer)

REPAIR
{'code': 0, 'message': 'Pyk, pyk, pyk, pytk jako tako i fajrant! Dałem z siebie całe 30% - proszę: IMG_1443_FT12.PNG'}
Downloaded repaired image IMG_1443_FT12.PNG


In [42]:
answer_1 = analyze_image("./img/IMG_559_NRR7.PNG")
print(answer_1)
answer_2 = analyze_image("./img/IMG_1410_FXER.PNG")
print(answer_2)
answer_3 = analyze_image("./img/IMG_1443_FT12.PNG")
print(answer_3)

The image shows a woman with long black hair and glasses. She is wearing a gray t-shirt and holding a coffee cup. She appears to be outdoors in an urban setting, standing near some greenery, and looking thoughtfully upward.
The image shows two women sitting side by side. The woman on the left has straight, red hair and is wearing glasses and a gray shirt. She appears to be looking thoughtfully into the distance. The woman on the right has long, dark hair, also wears glasses, and a gray shirt. She is looking in a similar direction. Both have a calm and contemplative expression.
The image shows a woman with long black hair and glasses. She is wearing a gray shirt and has a tattoo of an ant on her arm. She appears to be in a gym, holding onto exercise equipment.


In [96]:
def describe_person(image_urls: list[str]):

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": '''You get a images urls of Barbara and need to response with a name of the Barbara. 
                Barbara is a woman. 
                If picture is damaged reply with REPAIR. 
                To Dark reply with one word BRIGHTEN.
                To Brighten reply with one word DARKEN.
                If You can describe Barbara, reply with her description.
                think step by step. how she look describe details like hair, eyes, clothes, etc.
                what she is doing? what she have?
                From three images, describe one who is on all three images.
                Dont use general description but be precise. like not dark hair, but brown hair.
                Focus on distinctive features
                What other characteristic features does Barbara have in her appearance?
                What color is Barbara's hair?
                think twice about tatoo description, be specific.
                Dont talk about each photo, create one description of a person who is on all three images.
                Think. Describe where precisely is tattoo.
                Reply in Polish.
                '''
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_urls[0]
                        }
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_urls[1]
                        }
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_urls[2]
                        }
                    },
                    {
                        "type": "text", 
                        "text": "What is on this image?"
                    }
                ]
            }
        ],
        max_tokens=300
    )
    return response.choices[0].message.content

In [97]:
answer = describe_person(["https://centrala.ag3nts.org/dane/barbara/IMG_559_NRR7.PNG", "https://centrala.ag3nts.org/dane/barbara/IMG_1410_FXER.PNG", "https://centrala.ag3nts.org/dane/barbara/IMG_1443_FT12.PNG"])
print(answer)

Na wszystkich trzech zdjęciach widoczna jest kobieta o ciemnych, długich włosach. Nosi okulary oraz szary T-shirt. Wyróżnia się tatuażem na lewym ramieniu w formie czarnego pająka. Na pierwszym zdjęciu trzyma kubek, na ostatnim uchwyt maszyny do ćwiczeń.


In [98]:
answer_json = {
    "task": "photos",
    "apikey": personal_api_key,
    "answer": answer
}
answer_url = "https://centrala.ag3nts.org/report"
answer_response = requests.post(answer_url, json=answer_json)
print(answer_response.request.body.decode('unicode_escape'))  # Displays the request body decoded

# Assuming `login_response` is the response object from the login request
print("Status Code:", answer_response.status_code)  # Displays the status code (e.g., 200)
print("Response Body:", answer_response.text.encode('utf-8').decode('unicode_escape'))  # Display response content directly without encoding

{"task": "photos", "apikey": "1400cbf0-b7dd-49ab-9342-6ad8fd26ba69", "answer": "Na wszystkich trzech zdjęciach widoczna jest kobieta o ciemnych, długich włosach. Nosi okulary oraz szary T-shirt. Wyróżnia się tatuażem na lewym ramieniu w formie czarnego pająka. Na pierwszym zdjęciu trzyma kubek, na ostatnim uchwyt maszyny do ćwiczeń."}
Status Code: 200
Response Body: {
    "code": 0,
    "message": "{{FLG:USEFULCLUE}}"
}
